***GENERATED CODE FOR PredictCreditRisk PIPELINE***

**CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(inStages, inStagesData, stageId, spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(inStages, inStagesData, stageId, spark, config):
        return inStagesData.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                                        delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA**

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_transform"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    indexed = indexed.drop(feature).withColumnRenamed(outcol, feature)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        feature).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            feature, dfReturn[feature].cast(IntegerType()))
        return changed_type_df
    # changed_type_df.show(3)
    return dfReturn


Feature_Transformations_Methods = {
    "String Indexer": StringIndexerTransform,

}


class TransformationMain:
    # TODO: change df argument in run with following
    def run(inStages, inStagesData, stageId, spark, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = inStagesData[inStages[0]]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        for transformation in featureData:
            feature = transformation["feature"]
            if transformation["transformation"] != '' and transformation["selected"].lower() == "true" and not (feature.__contains__("_transform")):
                transformationDF = Feature_Transformations_Methods["%s" % transformation["transformation"]](
                    transformationDF, transformation)

        display(transformationDF.limit(2).toPandas())
        return transformationDF


**AUTOML FUNCTIONS**

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    df.columns.intersection(listOfFeatures)
    X = df.drop(label, axis=1).values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=15,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

try: 
	PredictCredit_Risk_DBFS = DBFSConnector.fetch([], {}, "5ea7b01f5290fb74ae38fdb2", spark, "{'url': '/Demo/PredictCreditRiskTrain.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
try: 
	PredictCredit_Risk_AutoFE = TransformationMain.run(["5ea7b01f5290fb74ae38fdb2"],{"5ea7b01f5290fb74ae38fdb2": PredictCredit_Risk_DBFS}, "5ea7b01f5290fb74ae38fdb3", spark,json.dumps( {"FE": [{"transformationsData": {"feature_label": "Status_of_checking_account"}, "feature": "Status_of_checking_account", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "362", "mean": "", "stddev": "", "min": "A11", "max": "A14", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "Duration_in_months", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "362", "mean": "21.09", "stddev": "12.46", "min": "4", "max": "60", "missing": "0"}}, {"transformationsData": {"feature_label": "Credit_history"}, "feature": "Credit_history", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "362", "mean": "", "stddev": "", "min": "A30", "max": "A34", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "Purpose"}, "feature": "Purpose", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "362", "mean": "", "stddev": "", "min": "A40", "max": "A49", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "Credit_amount", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "362", "mean": "3332.32", "stddev": "3015.15", "min": "250", "max": "18424", "missing": "0"}}, {"transformationsData": {"feature_label": "Savings_account_bond"}, "feature": "Savings_account_bond", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "362", "mean": "", "stddev": "", "min": "A61", "max": "A65", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "Present_employment_since"}, "feature": "Present_employment_since", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "362", "mean": "", "stddev": "", "min": "A71", "max": "A75", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "Installment_rate_in_percentage_of_disposable_income", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "362", "mean": "3.04", "stddev": "1.1", "min": "1", "max": "4", "missing": "0"}}, {"transformationsData": {"feature_label": "Personal_status_and_sex"}, "feature": "Personal_status_and_sex", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "362", "mean": "", "stddev": "", "min": "A91", "max": "A94", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "Other_debtors"}, "feature": "Other_debtors", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "362", "mean": "", "stddev": "", "min": "A101", "max": "A103", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "Present_residence_since", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "362", "mean": "2.87", "stddev": "1.09", "min": "1", "max": "4", "missing": "0"}}, {"transformationsData": {"feature_label": "Property"}, "feature": "Property", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "362", "mean": "", "stddev": "", "min": "A121", "max": "A124", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "Age_in_years", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "362", "mean": "35.82", "stddev": "11.71", "min": "19", "max": "75", "missing": "0"}}, {"transformationsData": {"feature_label": "Other_installment_plans"}, "feature": "Other_installment_plans", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "362", "mean": "", "stddev": "", "min": "A141", "max": "A143", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "Housing"}, "feature": "Housing", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "362", "mean": "", "stddev": "", "min": "A151", "max": "A153", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "Number_of_existing_credits", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "362", "mean": "1.4", "stddev": "0.59", "min": "1", "max": "4", "missing": "0"}}, {"transformationsData": {"feature_label": "Job"}, "feature": "Job", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "362", "mean": "", "stddev": "", "min": "A171", "max": "A174", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "Number_of_people", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "362", "mean": "1.14", "stddev": "0.35", "min": "1", "max": "2", "missing": "0"}}, {"transformationsData": {"feature_label": "Telephone"}, "feature": "Telephone", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "362", "mean": "", "stddev": "", "min": "A191", "max": "A192", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "Foreign_worker"}, "feature": "Foreign_worker", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "362", "mean": "", "stddev": "", "min": "A201", "max": "A202", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "Credit_Risk", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "362", "mean": "0.7", "stddev": "0.46", "min": "0", "max": "1", "missing": "0"}}, {"feature": "Status_of_checking_account_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "362", "mean": "1.05", "stddev": "0.98", "min": "0", "max": "3", "missing": "0"}}, {"feature": "Credit_history_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "362", "mean": "0.76", "stddev": "1.05", "min": "0.0", "max": "4.0", "missing": "0"}}, {"feature": "Purpose_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "362", "mean": "1.89", "stddev": "1.89", "min": "0.0", "max": "9.0", "missing": "0"}}, {"feature": "Savings_account_bond_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "362", "mean": "0.74", "stddev": "1.11", "min": "0.0", "max": "4.0", "missing": "0"}}, {"feature": "Present_employment_since_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "362", "mean": "1.38", "stddev": "1.26", "min": "0.0", "max": "4.0", "missing": "0"}}, {"feature": "Personal_status_and_sex_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "362", "mean": "0.59", "stddev": "0.8", "min": "0", "max": "3", "missing": "0"}}, {"feature": "Other_debtors_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "362", "mean": "0.14", "stddev": "0.47", "min": "0", "max": "2", "missing": "0"}}, {"feature": "Property_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "362", "mean": "1.23", "stddev": "1.08", "min": "0", "max": "3", "missing": "0"}}, {"feature": "Other_installment_plans_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "362", "mean": "0.23", "stddev": "0.52", "min": "0", "max": "2", "missing": "0"}}, {"feature": "Housing_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "362", "mean": "0.44", "stddev": "0.71", "min": "0", "max": "2", "missing": "0"}}, {"feature": "Job_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "362", "mean": "0.57", "stddev": "0.83", "min": "0", "max": "3", "missing": "0"}}, {"feature": "Telephone_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "362", "mean": "0.41", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}}, {"feature": "Foreign_worker_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "362", "mean": "0.03", "stddev": "0.18", "min": "0", "max": "1", "missing": "0"}}]}))

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
try: 
	functionClassification(PredictCredit_Risk_AutoFE,"5ea7b01f5290fb74ae38fdb3", ["Status_of_checking_account", "Duration_in_months", "Credit_history", "Purpose", "Credit_amount", "Savings_account_bond", "Present_employment_since", "Installment_rate_in_percentage_of_disposable_income", "Personal_status_and_sex", "Other_debtors", "Present_residence_since", "Property", "Age_in_years", "Other_installment_plans", "Housing", "Number_of_existing_credits", "Job", "Number_of_people", "Telephone", "Foreign_worker"], "Credit_Risk")

except Exception as ex: 
	logging.error(ex)
